In [1]:
using Laplacians

objc[35457]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[35457]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[35457]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[35457]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("../../../src/sparsify.jl")

sparsify (generic function with 1 method)

In [3]:
n = 10000;

In [4]:
a = chimera(n,23);

In [7]:
@time sp = sparsify(a, 0.5, cnTol=1e-1, verbose=true);

5856 23424
Trying a sparsifier with 5856 edges
Got condition number between (1.0000000000000395,1.1111111111111598)
  0.256220 seconds (758.04 k allocations: 76.962 MB, 2.73% gc time)


In [25]:
@time ldltfact(lap(chimera(30000,1)));

  0.861750 seconds (699.89 k allocations: 111.419 MB, 23.84% gc time)


In [26]:
?ldltfact

search: ldltfact ldltfact!



```
ldltfact(::Union{SparseMatrixCSC,Symmetric{Float64,SparseMatrixCSC{Flaot64,SuiteSparse_long}},Hermitian{Complex{Float64},SparseMatrixCSC{Complex{Float64},SuiteSparse_long}}}; shift=0, perm=Int[]) -> CHOLMOD.Factor
```

Compute the `LDLt` factorization of a sparse symmetric or Hermitian matrix. A fill-reducing permutation is used. `F = ldltfact(A)` is most frequently used to solve systems of equations `A*x = b` with `F\b`, but also the methods `diag`, `det`, `logdet` are defined for `F`. You can also extract individual factors from `F`, using `F[:L]`. However, since pivoting is on by default, the factorization is internally represented as `A == P'*L*D*L'*P` with a permutation matrix `P`; using just `L` without accounting for `P` will give incorrect answers. To include the effects of permutation, it's typically preferable to extact "combined" factors like `PtL = F[:PtL]` (the equivalent of `P'*L`) and `LtP = F[:UP]` (the equivalent of `L'*P`). The complete list of supported factors is `:L, :PtL, :D, :UP, :U, :LD, :DU, :PtLD, :DUP`.

Setting optional `shift` keyword argument computes the factorization of `A+shift*I` instead of `A`. If the `perm` argument is nonempty, it should be a permutation of `1:size(A,1)` giving the ordering to use (instead of CHOLMOD's default AMD ordering).

The function calls the C library CHOLMOD and many other functions from the library are wrapped but not exported.

```
ldltfact(::SymTridiagonal) -> LDLt
```

Compute an `LDLt` factorization of a real symmetric tridiagonal matrix such that `A = L*Diagonal(d)*L'` where `L` is a unit lower triangular matrix and `d` is a vector. The main use of an `LDLt` factorization `F = ldltfact(A)` is to solve the linear system of equations `Ax = b` with `F\b`.


In [35]:
isposdef(lap(a))

false

In [47]:
?ldltfact

search: ldltfact ldltfact!



```
ldltfact(::Union{SparseMatrixCSC,Symmetric{Float64,SparseMatrixCSC{Flaot64,SuiteSparse_long}},Hermitian{Complex{Float64},SparseMatrixCSC{Complex{Float64},SuiteSparse_long}}}; shift=0, perm=Int[]) -> CHOLMOD.Factor
```

Compute the `LDLt` factorization of a sparse symmetric or Hermitian matrix. A fill-reducing permutation is used. `F = ldltfact(A)` is most frequently used to solve systems of equations `A*x = b` with `F\b`, but also the methods `diag`, `det`, `logdet` are defined for `F`. You can also extract individual factors from `F`, using `F[:L]`. However, since pivoting is on by default, the factorization is internally represented as `A == P'*L*D*L'*P` with a permutation matrix `P`; using just `L` without accounting for `P` will give incorrect answers. To include the effects of permutation, it's typically preferable to extact "combined" factors like `PtL = F[:PtL]` (the equivalent of `P'*L`) and `LtP = F[:UP]` (the equivalent of `L'*P`). The complete list of supported factors is `:L, :PtL, :D, :UP, :U, :LD, :DU, :PtLD, :DUP`.

Setting optional `shift` keyword argument computes the factorization of `A+shift*I` instead of `A`. If the `perm` argument is nonempty, it should be a permutation of `1:size(A,1)` giving the ordering to use (instead of CHOLMOD's default AMD ordering).

The function calls the C library CHOLMOD and many other functions from the library are wrapped but not exported.

```
ldltfact(::SymTridiagonal) -> LDLt
```

Compute an `LDLt` factorization of a real symmetric tridiagonal matrix such that `A = L*Diagonal(d)*L'` where `L` is a unit lower triangular matrix and `d` is a vector. The main use of an `LDLt` factorization `F = ldltfact(A)` is to solve the linear system of equations `Ax = b` with `F\b`.


In [85]:
la = lap(a) + speye(n)
ldl = ldltfact(la);

In [78]:
?ldl

search: ldl ldltfact ldltfact! foldl mapfoldl Libdl InlineDisplay



No documentation found.

`ldl` is of type `Base.SparseMatrix.CHOLMOD.Factor{Float64}`:

**Summary:**

```julia
type Base.SparseMatrix.CHOLMOD.Factor{Float64} <: Factorization{Float64}
```

**Fields:**

```julia
p :: Ptr{Base.SparseMatrix.CHOLMOD.C_Factor{Float64}}
```


In [79]:
ldl[:UP]

Base.SparseMatrix.CHOLMOD.FactorComponent{Float64,:UP}
type:         LDLt
method: simplicial
maxnnz:     153716
nnz:        153716


In [95]:
d = 1 ./ (ldl[:D] \ ones(n));
d = spdiagm(d);

In [86]:
b = rand(n); b = b - norm(b);

In [96]:
x = ldl[:UP] \ (d \ (ldl[:PtL] \ b))

10000-element Array{Float64,1}:
 -57.2097
 -57.1982
 -57.0314
 -57.2329
 -57.2074
 -57.1006
 -56.9697
 -57.067 
 -57.1572
 -57.2308
 -57.2049
 -56.9963
 -57.148 
   ⋮     
 -57.1183
 -56.9274
 -57.1561
 -57.1667
 -57.1879
 -57.2712
 -57.1183
 -57.2965
 -57.0781
 -57.1829
 -57.2686
 -57.1155

In [97]:
norm((lap(a) + speye(n)) * x - b)

7.817717187280283e-12